In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.master("local[4]") \
.appName("Case_Situations") \
.config("spark.driver.memory","8g") \
.config("spark.executor.memory","8g") \
.getOrCreate()

22/11/13 05:30:09 WARN Utils: Your hostname, batuhan-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/11/13 05:30:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/13 05:30:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/13 05:30:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
data = [("Selin","Science",80,"P",90),
                ("Batu","Math",90,"P",70),
                ("Harun","English",20,"F",70),
                ("Burcu","Science",45,"F",75),
                ("Onur","Math",30,"F",50),
                ("Seda","Math",None,"NA",70),
                ]

student_schema = ["Name","Subject","Mark","Status","Attendance"]

df = spark.createDataFrame(data = data, schema = student_schema)

In [4]:
df.show()

+-----+-------+----+------+----------+
| Name|Subject|Mark|Status|Attendance|
+-----+-------+----+------+----------+
|Selin|Science|  80|     P|        90|
| Batu|   Math|  90|     P|        70|
|Harun|English|  20|     F|        70|
|Burcu|Science|  45|     F|        75|
| Onur|   Math|  30|     F|        50|
| Seda|   Math|null|    NA|        70|
+-----+-------+----+------+----------+



# Update the existing Column

In [5]:
from pyspark.sql.functions import when
df1 = df.withColumn("Status", when(df.Mark >= 50, "Pass")
                   .when(df.Mark <50, "Fail")
                   .otherwise("Absente"))
df1.show()

+-----+-------+----+-------+----------+
| Name|Subject|Mark| Status|Attendance|
+-----+-------+----+-------+----------+
|Selin|Science|  80|   Pass|        90|
| Batu|   Math|  90|   Pass|        70|
|Harun|English|  20|   Fail|        70|
|Burcu|Science|  45|   Fail|        75|
| Onur|   Math|  30|   Fail|        50|
| Seda|   Math|null|Absente|        70|
+-----+-------+----+-------+----------+



# Create a New Column

In [6]:
df2 = df.withColumn("New Status", when(df.Mark >= 50, "Pass")
                   .when(df.Mark < 50, "Fail")
                   .otherwise("Absentee"))
df2.show()

+-----+-------+----+------+----------+----------+
| Name|Subject|Mark|Status|Attendance|New Status|
+-----+-------+----+------+----------+----------+
|Selin|Science|  80|     P|        90|      Pass|
| Batu|   Math|  90|     P|        70|      Pass|
|Harun|English|  20|     F|        70|      Fail|
|Burcu|Science|  45|     F|        75|      Fail|
| Onur|   Math|  30|     F|        50|      Fail|
| Seda|   Math|null|    NA|        70|  Absentee|
+-----+-------+----+------+----------+----------+



# Multi-Conditions AND & OR Operators 

# ==> | operator at least one case should provide

# ==> & operator bot of cases should provide

In [7]:
df3 = df.withColumn("Grade", when((df.Mark >= 90) | (df.Attendance >= 90), "Very Good")
                    .when((df.Mark >= 50) & (df.Attendance >=70), "Good")
                    .otherwise("Average"))

df3.show()

+-----+-------+----+------+----------+---------+
| Name|Subject|Mark|Status|Attendance|    Grade|
+-----+-------+----+------+----------+---------+
|Selin|Science|  80|     P|        90|Very Good|
| Batu|   Math|  90|     P|        70|Very Good|
|Harun|English|  20|     F|        70|  Average|
|Burcu|Science|  45|     F|        75|  Average|
| Onur|   Math|  30|     F|        50|  Average|
| Seda|   Math|null|    NA|        70|  Average|
+-----+-------+----+------+----------+---------+



In [8]:
df4 = df.withColumn("Grade", when((df.Mark >= 80) & (df.Attendance >= 80), "Very Good")
                    .when((df.Mark >= 50) | (df.Attendance >=50), "Good")
                    .otherwise("Average"))

df4.show()

+-----+-------+----+------+----------+---------+
| Name|Subject|Mark|Status|Attendance|    Grade|
+-----+-------+----+------+----------+---------+
|Selin|Science|  80|     P|        90|Very Good|
| Batu|   Math|  90|     P|        70|     Good|
|Harun|English|  20|     F|        70|     Good|
|Burcu|Science|  45|     F|        75|     Good|
| Onur|   Math|  30|     F|        50|     Good|
| Seda|   Math|null|    NA|        70|     Good|
+-----+-------+----+------+----------+---------+

